In [284]:
import os
import sys
module_path = os.path.abspath(os.path.join('../..'))
print(module_path)
if module_path not in sys.path:
    sys.path.append(module_path)
import pandas as pd
import numpy as np
from src import parse
from collections import Counter

/Users/andrew/src/otp-scheduler


In [307]:
def sort_schedule(df):
    return df.sort_values(by=['zone', 'priority_rank', 'code']).reset_index(drop=True)

restaurants = parse.parse_from_restaurants_csv('../../examples/boston/20200415_no_bf/restaurants.csv').set_index('code')
hospitals = sort_schedule(parse.parse_from_hospital_requests_csv('../../examples/boston/20200415_no_bf/hospitals.csv')).set_index('name')
vip_hospitals = sort_schedule(parse.parse_from_hospital_requests_csv('../../examples/boston/20200415_no_bf/vips.csv')).set_index('name')
vip_schedule = vip_hospitals.iloc[:, 4:]

rest_usage = restaurants.copy()
rest_usage.iloc[:, 2:] = 0

rest_cap = restaurants.copy()

# ASSUMPTIONS
MEAL_CAPACITY = 20000
MAX_DROPOFFS_PER_RUN = 4
MIN_MEALS_PER_RUN = 50

curr_capacity = MEAL_CAPACITY

total_hospitals = hospitals.shape[0]

### Idea #1

1. Go in order of hospital priority + who has least meals served so far (start with VIPs?)
2. find the order request that’s the most meals (or a request that’s in the biggest unserved cluster) that hasn’t yet been fulfilled. 
3. Assign a restaurant that can handle that restaurant and hopefully everything else in the cluster. 
4. Adjust meal counts for other hospitals that will be delivered to in that cluster. 
5. Update meals for all hospitals, capacity for that restaurant, and repeat. 

In [308]:
def day_from_meal(meal):
    return meal.replace('lunch', '').replace('breakfast', '').replace('dinner', '').replace('_', '')

meal_log = None
meal_log_key_kludge = set()

def get_meal_log(schedule, is_vip):
    log = []
    for name, row in schedule.iterrows():
        for j, val in enumerate(row[4:]):
            # Add only if this cell is not blank, AND we haven't already added this hospital/meal.
            # This is possible if this order exists in both the regular and VIP meal schedule.
            # Since we add the VIP meals first, then we're guaranteed for it to be counted as a VIP meal if it's in both places.
            if not pd.isnull(val) and (name, schedule.columns[j + 4]) not in meal_log_key_kludge:
                log.append([
                    name,
                    row['zone'],
                    row['priority_rank'],
                    is_vip,
                    schedule.columns[j + 4],
                    val, 
                    np.nan,
                    day_from_meal(schedule.columns[j + 4]),
                    False
                ])
                meal_log_key_kludge.add((name, schedule.columns[j + 4]))
    return pd.DataFrame(log, columns=['hospital', 'zone', 'priority_rank', 'vip', 'meal', 'quantity', 'restaurant', 'day', 'failed'])

vip_log = get_meal_log(vip_hospitals, True)
meal_log = get_meal_log(hospitals, False).append(vip_log)

In [313]:
meal_log.sample(frac=1).reset_index().iloc[:, 1:]

,hospital,zone,priority_rank,vip,meal,quantity,restaurant,day,failed
0,BWH - Faulkner ICU,2,1,True,lunch_sun,17.0,NaN,sun,False
1,Steward St. Elizabeth's Medical Center,5,1,False,lunch_mon,50.0,NaN,mon,False
2,Boston Healthcare for the Homeless - Albany St...,2,1,True,dinner_mon,20.0,NaN,mon,False
3,BWH - Faulkner ICU,2,1,True,lunch_wed,17.0,NaN,wed,False
4,Boston Healthcare for the Homeless - Tents,2,1,True,dinner_thurs,12.0,NaN,thurs,False
...,...,...,...,...,...,...,...,...,...
265,Cambridge Health Alliance - Cambridge Hospital,4,1,False,lunch_tues,115.0,NaN,tues,False
266,Brigham & Women's Hospital (COVID ICUs),1,1,True,lunch_fri,210.0,NaN,fri,False
267,Cambridge Health Alliance - Everett Hospital,3,1,False,lunch_sun,75.0,NaN,sun,False
268,Boston Medical Center,2,1,False,dinner_thurs,400.0,NaN,thurs,False


In [288]:
# 1. Fill outstanding VIP meals
# 2. Fill out with LCD approach (select min, if tie take one with better priority)

In [289]:
def maybe_compact(df, meal, compact):
    if compact:
        return df[df[meal].notnull()]
    return df

def get_rest_cap_cluster_for_meal(meal, compact=False):
    r = rest_cap[[meal]]
    return maybe_compact(r, meal, compact)

In [290]:
def get_oustanding_vip_meals():
    return meal_log[(meal_log['vip'] == True) & meal_log['restaurant'].isnull()]

def get_outstanding_hospitals_in_cluster_for_meal(meal, zone):
    return meal_log[(meal_log['meal'] == meal) & (meal_log['zone'] == zone) & meal_log['restaurant'].isnull()]

def get_hospitals_in_cluster_for_meal(meal, zone):
    return meal_log[(meal_log['meal'] == meal) & (meal_log['zone'] == zone)]

def is_restaurant_serving_same_day_other_meal(rest_code, meal):
    day = day_from_meal(meal)
    rest_serving = meal_log[(meal_log['restaurant'] == rest_code) & (meal_log['day'] == day) & meal_log['meal'] != meal]
    return not rest_serving.empty

def assign_hospital_order(row):
    global curr_capacity

    meal = row['meal']
    hospital = row['hospital']
    
    print('Assigning', hospital, meal)
    
    # Check to see if we've already done this (sanity)
    if not pd.isnull(row['restaurant']):
        print('We have already assigned this one!')
        return False
        
    hospitals_in_cluster = get_hospitals_in_cluster_for_meal(meal, row['zone'])

    # Restaurant capacity
    r_cap = get_rest_cap_cluster_for_meal(meal)
        
    # The amount have to serve
    target_quantity = row['quantity']
    
    if curr_capacity - target_quantity < 0:
        print('We cannot to afford this meal because of our global limit!', curr_capacity, target_quantity)
        return False
    
    print(hospital, meal, row['zone'], target_quantity)
    
    def find_restaurant():
        # Check if a restaurant is already serving the cluster
        restaurants_serving_cluster = hospitals_in_cluster[hospitals_in_cluster['restaurant'].notnull()]['restaurant']
        
        if not restaurants_serving_cluster.empty:
            # Filter out any that are already at the dropout limit
            rest_counter = Counter(restaurants_serving_cluster)
            rest_cluster_keys = rest_counter.keys()
            print('Checking for restaurants already in cluster', rest_counter)
            rest_with_room = [c[0] for c in rest_counter.most_common() if c[1] < MAX_DROPOFFS_PER_RUN]
            
            # Find restaurants with capacity
            candidates_with_quantity = []
            for r in rest_with_room:
                q = rest_cap.loc[r, meal]
                if q >= target_quantity:
                    print('Found an existing restaurant with capacity', r, q)
                    candidates_with_quantity.append(r)
                    if is_restaurant_serving_same_day_other_meal(r, meal):
                        print('Returning a restaurant with capacity, working on the same day', r, q)
                        return r
            if len(candidates_with_quantity):
                print('Returning a restaurant with capacity', r, q)
                return candidates_with_quantity[0]
        
        print('No suitable restaurants already working this cluster')
        # Couldn't find an appropriate restaurant already serving cluster
        
        restaurants_in_other_zones = meal_log[(meal_log['meal'] == meal) & (meal_log['zone'] != row['zone']) & (meal_log['restaurant'].notnull())]['restaurant'].unique()
        
        print('filtering our candidates with two lists', restaurants_serving_cluster, restaurants_in_other_zones)
        r_candidates = r_cap[(~r_cap.index.isin(restaurants_serving_cluster)) & (~r_cap.index.isin(restaurants_in_other_zones)) & (r_cap[meal] >= target_quantity)]
        
        print('Number of candidates with capacity', r_candidates.shape[0])
        
        if r_candidates.empty:
            return None
        
        # TODO add is_restaurant_serving_same_day_other_meal logic down here too!
        
        # Sort in quantity order (TODO KNOB: THIS USED TO BE ascending=False)
        r_candidates = r_candidates.sort_values(by=meal, ascending=True)
        
        day = day_from_meal(meal)
        
        # TODO Make sure restaurant isn't overbooked on their max day count
        final_valid_list = []
        for i, r in r_candidates.iterrows():
            print(i)
            days_worked = set(meal_log[meal_log['restaurant'] == i]['day'])
            # Check if we're already working this day, or the new day is still under cap
            if day in days_worked or len(days_worked) < rest_cap.loc[i, 'max_days_per_week']:
                # return i
                final_valid_list.append(i)
        
        # MEAL STEAL -- We need to prioritize founding Restaurants (PG, LD)
        # TODO -- parameterize this, both the restaurants we want to do this for, and also how much we crank this knob
        # Comment out this section to ignore
        meal_steal_knob = 3
        if len(final_valid_list) == 0:
            return None
        else:
            if len(final_valid_list) == 1:
                return final_valid_list[0]
            else:
                for r in final_valid_list[0:meal_steal_knob]:
                    if r in ('PG', 'LD'):
                        return r
                return final_valid_list[0]
        
        return None
          
    r_code = find_restaurant()
    
    if r_code is None:
        print('CANNOT SERVE HOSPITAL FOR THEIR REQUEST', hospital, meal, r_cap)
        return False
                
    # Allocate the restaurant to the hospital(s)
    print('debiting r capacity: ', r_code, rest_cap.loc[r_code, meal])
    rest_cap.loc[r_code, meal] -= target_quantity
            
    print('increasing r usage', r_code, target_quantity)
    rest_usage.loc[r_code, meal] += target_quantity
        
    curr_capacity -= target_quantity
                
    print('updating log', hospital, meal, r_code)
    meal_log.loc[(meal_log['hospital'] == hospital) & (meal_log['meal'] == meal), 'restaurant'] = r_code
    
    return True

# HANDLE VIPS
for i, row in get_oustanding_vip_meals().iterrows():
    assigned = assign_hospital_order(row)
    if not assigned:
        meal_log.loc[row['hospital'], 'failed'] = True
    print()
    print() 

Assigning BIDMC - West Campus lunch_mon
BIDMC - West Campus lunch_mon 1 45.0
No suitable restaurants already working this cluster
filtering our candidates with two lists Series([], Name: restaurant, dtype: float64) []
Number of candidates with capacity 10
RT
IG
PO
SH
FC
MD
BB
SU
PG
LD
debiting r capacity:  RT 150.0
increasing r usage RT 45.0
updating log BIDMC - West Campus lunch_mon RT


Assigning BIDMC - West Campus lunch_tues
BIDMC - West Campus lunch_tues 1 45.0
No suitable restaurants already working this cluster
filtering our candidates with two lists Series([], Name: restaurant, dtype: object) []
Number of candidates with capacity 10
ME
IG
PO
SH
FC
MD
BB
SU
PG
LD
debiting r capacity:  ME 100.0
increasing r usage ME 45.0
updating log BIDMC - West Campus lunch_tues ME


Assigning BIDMC - West Campus lunch_wed
BIDMC - West Campus lunch_wed 1 45.0
No suitable restaurants already working this cluster
filtering our candidates with two lists Series([], Name: restaurant, dtype: object) 

/Users/andrew/.pyenv/versions/3.8.0/lib/python3.8/site-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)



Assigning BIDMC - East Campus lunch_thurs
BIDMC - East Campus lunch_thurs 1 30.0
Checking for restaurants already in cluster Counter({'ME': 1})
Found an existing restaurant with capacity ME 55.0
Returning a restaurant with capacity, working on the same day ME 55.0
debiting r capacity:  ME 55.0
increasing r usage ME 30.0
updating log BIDMC - East Campus lunch_thurs ME


Assigning BIDMC - East Campus lunch_fri
BIDMC - East Campus lunch_fri 1 30.0
Checking for restaurants already in cluster Counter({'ME': 1})
Found an existing restaurant with capacity ME 55.0
Returning a restaurant with capacity, working on the same day ME 55.0
debiting r capacity:  ME 55.0
increasing r usage ME 30.0
updating log BIDMC - East Campus lunch_fri ME


Assigning BIDMC - East Campus lunch_sat
BIDMC - East Campus lunch_sat 1 6.0
Checking for restaurants already in cluster Counter({'IG': 1})
Found an existing restaurant with capacity IG 155.0
Returning a restaurant with capacity, working on the same day IG 155.0


Assigning Brigham & Women's Hospital (COVID ICUs) dinner_sat
Brigham & Women's Hospital (COVID ICUs) dinner_sat 1 200.0
No suitable restaurants already working this cluster
filtering our candidates with two lists Series([], Name: restaurant, dtype: object) []
Number of candidates with capacity 6
SH
MD
BB
LD
SU
PG
debiting r capacity:  SH 300.0
increasing r usage SH 200.0
updating log Brigham & Women's Hospital (COVID ICUs) dinner_sat SH


Assigning Brigham & Women's Hospital (COVID ICUs) dinner_sun
Brigham & Women's Hospital (COVID ICUs) dinner_sun 1 200.0
Checking for restaurants already in cluster Counter({'MM': 1})
No suitable restaurants already working this cluster
filtering our candidates with two lists 17    MM
Name: restaurant, dtype: object []
Number of candidates with capacity 6
SH
MD
BB
LD
SU
PG
debiting r capacity:  SH 300.0
increasing r usage SH 200.0
updating log Brigham & Women's Hospital (COVID ICUs) dinner_sun SH


Assigning Brigham and Women's - Faulkner ED lunch_wed


Assigning Boston Healthcare for the Homeless - Tents dinner_sun
Boston Healthcare for the Homeless - Tents dinner_sun 2 12.0
Checking for restaurants already in cluster Counter({'LD': 1})
Found an existing restaurant with capacity LD 455.0
Returning a restaurant with capacity, working on the same day LD 455.0
debiting r capacity:  LD 455.0
increasing r usage LD 12.0
updating log Boston Healthcare for the Homeless - Tents dinner_sun LD


Assigning Carney Hospital - ED dinner_mon
Carney Hospital - ED dinner_mon 2 100.0
Checking for restaurants already in cluster Counter({'FC': 1})
No suitable restaurants already working this cluster
filtering our candidates with two lists 47    FC
Name: restaurant, dtype: object ['SH']
Number of candidates with capacity 5
MD
BB
LD
SU
PG
debiting r capacity:  LD 500.0
increasing r usage LD 100.0
updating log Carney Hospital - ED dinner_mon LD


Assigning Carney Hospital - ED dinner_tues
Carney Hospital - ED dinner_tues 2 100.0
Checking for restaurants al

Found an existing restaurant with capacity IG 170.0
Returning a restaurant with capacity, working on the same day IG 170.0
debiting r capacity:  IG 170.0
increasing r usage IG 17.0
updating log BWH - Faulkner ICU lunch_mon IG


Assigning BWH - Faulkner ICU lunch_tues
BWH - Faulkner ICU lunch_tues 2 17.0
Checking for restaurants already in cluster Counter({'IG': 2, 'FC': 1})
Found an existing restaurant with capacity IG 170.0
Returning a restaurant with capacity, working on the same day IG 170.0
debiting r capacity:  IG 170.0
increasing r usage IG 17.0
updating log BWH - Faulkner ICU lunch_tues IG


Assigning BWH - Faulkner ICU lunch_wed
BWH - Faulkner ICU lunch_wed 2 17.0
Checking for restaurants already in cluster Counter({'IG': 3, 'FC': 1})
Found an existing restaurant with capacity IG 125.0
Returning a restaurant with capacity, working on the same day IG 125.0
debiting r capacity:  IG 125.0
increasing r usage IG 17.0
updating log BWH - Faulkner ICU lunch_wed IG


Assigning BWH - Fau


Assigning Tufts Medical Center lunch_thurs
Tufts Medical Center lunch_thurs 3 50.0
No suitable restaurants already working this cluster
filtering our candidates with two lists Series([], Name: restaurant, dtype: object) ['ME' 'PO' 'IG' 'FC' 'SH']
Number of candidates with capacity 5
MD
BB
SU
PG
LD
debiting r capacity:  MD 400.0
increasing r usage MD 50.0
updating log Tufts Medical Center lunch_thurs MD


Assigning Tufts Medical Center lunch_fri
Tufts Medical Center lunch_fri 3 50.0
No suitable restaurants already working this cluster
filtering our candidates with two lists Series([], Name: restaurant, dtype: object) ['ME' 'PO' 'RT' 'FC' 'IG']
Number of candidates with capacity 6
SH
MD
BB
SU
PG
LD
debiting r capacity:  SH 300.0
increasing r usage SH 50.0
updating log Tufts Medical Center lunch_fri SH


Assigning Tufts Medical Center dinner_mon
Tufts Medical Center dinner_mon 3 54.0
No suitable restaurants already working this cluster
filtering our candidates with two lists Series([], N

In [291]:
# meal_log[meal_log.vip == True]

In [292]:
hospitals_that_failed_placement = set()

def select_and_assign_hospital():
    next_hospital = meal_log[~meal_log.hospital.isin(list(hospitals_that_failed_placement))].groupby('hospital')[['priority_rank', 'restaurant']].agg({
            'restaurant': 'count',
            'priority_rank': 'min'
        }).sort_values(by=['restaurant', 'priority_rank']).reset_index().iloc[0]
    
    print('Next hospital:', next_hospital)

    # Find potential orders to serve, ordered by quantity
    possible_orders = meal_log[(meal_log['hospital'] == next_hospital['hospital']) & (meal_log['restaurant'].isnull()) & (meal_log['failed'] == False)].sort_values(by='quantity', ascending=False)
    
    print('Possible orders', possible_orders)
    for i, row in possible_orders.iterrows():
        if assign_hospital_order(row):
            return True
        else:
            meal_log.loc[
                (meal_log['hospital'] == row['hospital']) & (meal_log['meal'] == row['meal']), 
                'failed'
            ] = True
            
    hospitals_that_failed_placement.add(next_hospital['hospital'])
    

while curr_capacity > 0 and len(hospitals_that_failed_placement) < total_hospitals:
    select_and_assign_hospital()
    print()
    print()

Next hospital: hospital         Boston Children's Hospital - ED
restaurant                                     0
priority_rank                                  1
Name: 0, dtype: object
Possible orders                           hospital  zone  priority_rank    vip          meal  \
0  Boston Children's Hospital - ED     1              1  False    dinner_mon   
1  Boston Children's Hospital - ED     1              1  False   dinner_tues   
2  Boston Children's Hospital - ED     1              1  False    dinner_wed   
3  Boston Children's Hospital - ED     1              1  False  dinner_thurs   
4  Boston Children's Hospital - ED     1              1  False    dinner_fri   

   quantity restaurant    day  failed  
0      60.0        NaN    mon   False  
1      60.0        NaN   tues   False  
2      60.0        NaN    wed   False  
3      60.0        NaN  thurs   False  
4      60.0        NaN    fri   False  
Assigning Boston Children's Hospital - ED dinner_mon
Boston Children's Hospita

/Users/andrew/.pyenv/versions/3.8.0/lib/python3.8/site-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)
/Users/andrew/.pyenv/versions/3.8.0/lib/python3.8/site-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)
/Users/andrew/.pyenv/versions/3.8.0/lib/python3.8/site-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)
/Users/andrew/.pyenv/versions/3.8.0/lib/python3.8/site-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_value

 Dana Farber Cancer Institute - Inpatient PAs lunch_mon RT


Next hospital: hospital         Mt. Auburn Hospital
restaurant                         0
priority_rank                      1
Name: 0, dtype: object
Possible orders                 hospital  zone  priority_rank    vip        meal  quantity  \
101  Mt. Auburn Hospital     4              1  False  dinner_wed      75.0   

    restaurant  day  failed  
101        NaN  wed   False  
Assigning Mt. Auburn Hospital dinner_wed
Mt. Auburn Hospital dinner_wed 4 75.0
No suitable restaurants already working this cluster
filtering our candidates with two lists Series([], Name: restaurant, dtype: object) ['FC' 'SH' 'CS' 'ME' 'LD' 'MD' 'PG']
Number of candidates with capacity 2
BB
SU
debiting r capacity:  BB 400.0
increasing r usage BB 75.0
updating log Mt. Auburn Hospital dinner_wed BB


Next hospital: hospital         Newton Wellesley Hospital
restaurant                               0
priority_rank                            1
Name: 0, d

/Users/andrew/.pyenv/versions/3.8.0/lib/python3.8/site-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)
/Users/andrew/.pyenv/versions/3.8.0/lib/python3.8/site-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)
/Users/andrew/.pyenv/versions/3.8.0/lib/python3.8/site-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)
/Users/andrew/.pyenv/versions/3.8.0/lib/python3.8/site-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_value


Assigning Dimock Center lunch_tues
Dimock Center lunch_tues 2 20.0
Checking for restaurants already in cluster Counter({'IG': 4, 'FC': 1, 'SH': 1})
Found an existing restaurant with capacity SH 230.0
Returning a restaurant with capacity, working on the same day SH 230.0
debiting r capacity:  SH 230.0
increasing r usage SH 20.0
updating log Dimock Center lunch_tues SH


Next hospital: hospital         Dot House Health Dorchester
restaurant                                 0
priority_rank                              2
Name: 0, dtype: object
Possible orders                        hospital  zone  priority_rank    vip         meal  \
33  Dot House Health Dorchester     2              2  False    lunch_mon   
34  Dot House Health Dorchester     2              2  False   lunch_tues   
35  Dot House Health Dorchester     2              2  False    lunch_wed   
36  Dot House Health Dorchester     2              2  False  lunch_thurs   
37  Dot House Health Dorchester     2              2  Fals

...



Next hospital: hospital         Cambridge Health Alliance - Everett Hospital
restaurant                                                  6
priority_rank                                               1
Name: 0, dtype: object
Possible orders                                         hospital  zone  priority_rank    vip  \
71  Cambridge Health Alliance - Everett Hospital     3              1  False   

         meal  quantity restaurant  day  failed  
71  lunch_sun      75.0        NaN  sun   False  
Assigning Cambridge Health Alliance - Everett Hospital lunch_sun
Cambridge Health Alliance - Everett Hospital lunch_sun 3 75.0
Checking for restaurants already in cluster Counter({'LD': 1})
Found an existing restaurant with capacity LD 650.0
Returning a restaurant with capacity, working on the same day LD 650.0
debiting r capacity:  LD 650.0
increasing r usage LD 75.0
updating log Cambridge Health Alliance - Everett Hospital lunch_sun LD


Next hospital: hospital         Cambridge Health Alli

/Users/andrew/.pyenv/versions/3.8.0/lib/python3.8/site-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)
/Users/andrew/.pyenv/versions/3.8.0/lib/python3.8/site-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)
/Users/andrew/.pyenv/versions/3.8.0/lib/python3.8/site-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)


 Empty DataFrame
Columns: [hospital, zone, priority_rank, vip, meal, quantity, restaurant, day, failed]
Index: []


Next hospital: hospital         Carney Hospital - ED
restaurant                          7
priority_rank                       1
Name: 0, dtype: object
Possible orders Empty DataFrame
Columns: [hospital, zone, priority_rank, vip, meal, quantity, restaurant, day, failed]
Index: []


Next hospital: hospital         Dana Farber Cancer Institute - Inpatient PAs
restaurant                                                  7
priority_rank                                               1
Name: 0, dtype: object
Possible orders Empty DataFrame
Columns: [hospital, zone, priority_rank, vip, meal, quantity, restaurant, day, failed]
Index: []


Next hospital: hospital         Massachusetts General Hospital - Central
restaurant                                              7
priority_rank                                           1
Name: 0, dtype: object
Possible orders Empty DataFrame
Co

/Users/andrew/.pyenv/versions/3.8.0/lib/python3.8/site-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)
/Users/andrew/.pyenv/versions/3.8.0/lib/python3.8/site-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)
/Users/andrew/.pyenv/versions/3.8.0/lib/python3.8/site-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)


Next hospital: hospital         Tufts Medical Center
restaurant                         10
priority_rank                       1
Name: 0, dtype: object
Possible orders Empty DataFrame
Columns: [hospital, zone, priority_rank, vip, meal, quantity, restaurant, day, failed]
Index: []


Next hospital: hospital         BWH - Faulkner 7N Covid Unit
restaurant                                 14
priority_rank                               1
Name: 0, dtype: object
Possible orders Empty DataFrame
Columns: [hospital, zone, priority_rank, vip, meal, quantity, restaurant, day, failed]
Index: []


Next hospital: hospital         BWH - Faulkner ICU
restaurant                       14
priority_rank                     1
Name: 0, dtype: object
Possible orders Empty DataFrame
Columns: [hospital, zone, priority_rank, vip, meal, quantity, restaurant, day, failed]
Index: []


Next hospital: hospital         Boston Healthcare for the Homeless - Albany St...
restaurant                                         

In [293]:
curr_capacity, hospitals_that_failed_placement

(36.0,
 {'BIDMC - Chelsea',
  'BIDMC - East Campus',
  'BIDMC - ICU East',
  'BIDMC - ICU West',
  'BIDMC - West Campus',
  'BWH - Faulkner 7N Covid Unit',
  'BWH - Faulkner ICU',
  "Boston Children's Hospital - ED",
  'Boston Healthcare for the Homeless - Albany Street',
  'Boston Healthcare for the Homeless - Tents',
  'Boston Medical Center',
  "Brigham & Women's Hospital (COVID ICUs)",
  "Brigham and Women's - Faulkner ED",
  "Brigham and Women's Hospital - ED",
  'Cambridge Health Alliance - Cambridge Hospital',
  'Cambridge Health Alliance - Everett Hospital',
  'Cambridge Health Alliance - Somerville Hospital',
  'Carney Hospital - ED',
  'Charles River Community Health',
  'Codman Square Health Center - Site 1',
  'Codman Square Health Center - Site 2',
  'Dana Farber Cancer Institute - Inpatient PAs',
  'Dimock Center',
  'Dot House Health Dorchester',
  'East Boston Neighborhood Health Center',
  'Harvard Street Neighborhood Health Center',
  'Massachusetts General Hospital -

In [294]:
meal_log[meal_log['restaurant'].notnull()]['quantity'].sum()

19964.0

In [295]:
# Create empty results frame
output = hospitals.copy()
output.iloc[:, 4:] = np.nan


# construct the double-wide output dataframe
for i, c in enumerate(hospitals.columns[4:]):
    output.insert((i*2+1)+4, c + '_rest', np.nan)
        
for i, row in meal_log.iterrows():
    output.loc[row['hospital'], row['meal']] = row['quantity']
    if not pd.isnull(row['restaurant']):
        output.loc[row['hospital'], row['meal'] + '_rest'] = row['restaurant']
    else:
        output.loc[row['hospital'], row['meal'] + '_rest'] = 'XXX'


# for i, row in meal_log.iterrows():
#     if not pd.isnull(row['restaurant']):
#         val = str(row['restaurant'] + ' - ' + str(row['quantity']))
#     else:
#         val = str('XXX - ' + str(row['quantity']))
#     output.loc[row['hospital'], row['meal']] = val
    
# for i, row in meal_log.iterrows():
#     if not pd.isnull(row['restaurant']):
#         output.loc[row['hospital'], row['meal']] = row['restaurant']
        
output.to_csv('../../output.csv')


In [296]:
output

,code,priority_rank,region,zone,breakfast_mon,breakfast_mon_rest,breakfast_tues,breakfast_tues_rest,breakfast_wed,breakfast_wed_rest,...,dinner_wed,dinner_wed_rest,dinner_thurs,dinner_thurs_rest,dinner_fri,dinner_fri_rest,dinner_sat,dinner_sat_rest,dinner_sun,dinner_sun_rest
name,,,,,,,,,,,,,,,,,,,,,
BIDMC - West Campus,2,1,Longwood / Back Bay,1,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
BIDMC - East Campus,3,1,Longwood / Back Bay,1,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Brigham and Women's Hospital - ED,4,1,Longwood / Back Bay,1,NaN,NaN,NaN,NaN,NaN,NaN,...,100.0,FC,NaN,NaN,NaN,NaN,NaN,NaN,100.0,MM
Boston Children's Hospital - ED,18,1,Longwood / Back Bay,1,NaN,NaN,NaN,NaN,NaN,NaN,...,60.0,SH,60.0,SH,60.0,SH,NaN,NaN,NaN,NaN
BIDMC - ICU West,21,1,Longwood / Back Bay,1,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,60.0,FC,NaN,NaN,NaN,NaN,NaN,NaN
BIDMC - ICU East,22,1,Longwood / Back Bay,1,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,20.0,FC,NaN,NaN,NaN,NaN,NaN,NaN
Dana Farber Cancer Institute - Inpatient PAs,23,1,Longwood / Back Bay,1,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Brigham & Women's Hospital (COVID ICUs),36,1,Longwood / Back Bay,1,NaN,NaN,NaN,NaN,NaN,NaN,...,210.0,SH,210.0,SH,210.0,SH,200.0,SH,200.0,SH
Whittier Street Health Center,15,2,Longwood / Back Bay,1,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [297]:
rest_usage.iloc[:, 2:].sum(axis=1).sort_values(ascending=False).sum()

19964.0

In [298]:
hospitals.iloc[:, 4:].sum().sum()

22503.0

In [299]:
equity = meal_log.groupby(['priority_rank', 'hospital'])[['meal', 'vip', 'restaurant']].agg({
    'restaurant': [('orders_fulfilled', 'count')],
    'meal': [('orders_requested', 'count')],
    'vip': [('vip_orders', 'sum')],
})
equity

restaurant  \
                                                                 orders_fulfilled   
priority_rank hospital                                                              
1             BIDMC - East Campus                                               7   
              BIDMC - ICU East                                                  2   
              BIDMC - ICU West                                                  2   
              BIDMC - West Campus                                               7   
              BWH - Faulkner 7N Covid Unit                                     14   
              BWH - Faulkner ICU                                               14   
              Boston Children's Hospital - ED                                   5   
              Boston Healthcare for the Homeless - Albany Street               14   
              Boston Healthcare for the Homeless - Tents                       14   
              Boston Medical Center                                             9   
              Brigham & Women's Hospital (COVID ICUs)                          14   
              Brigham and Women's - Faulkner ED                                 4   
              Brigham and Women's Hospital - ED                                 4   
              Cambridge Health Alliance - Cambridge Hospital                    7   
              Cambridge Health Alliance - Everett Hospital                      7   
              Cambridge Health Alliance - Somerville Hospital                   7   
              Carney Hospital - ED                                              7   
              Dana Farber Cancer Institute - Inpatient PAs                      7   
              Massachusetts General Hospital - Central                          7   
              Massachusetts General Hospital - PICU                             7   
              Mt. Auburn Hospital                                               1   
              Newton Wellesley Hospital                                         9   
              Steward St. Elizabeth's Medical Center                            8   
              Tufts Medical Center                                             10   
              VA Medical Center - West Roxbury                                  6   
2             BIDMC - Chelsea                                                   5   
              Charles River Community Health                                    4   
              Codman Square Health Center - Site 1                              5   
              Codman Square Health Center - Site 2                              4   
              Dimock Center                                                     3   
              Dot House Health Dorchester                                       4   
              East Boston Neighborhood Health Center                            6   
              Harvard Street Neighborhood Health Center                         4   
              Mattapan Community Health Center                                  3   
              North End Waterfront Health Center                                3   
              South Boston Community Health Center                              5   
              Southern Jamaica Plain Health Center (part of B...                4   
              Upham's Corner Health Center                                      3   
              Whittier Street Health Center                                     4   

                                                                             meal  \
                                                                 orders_requested   
priority_rank hospital                                                              
1             BIDMC - East Campus                                               7   
              BIDMC - ICU East                                                  2   
              BIDMC - ICU West                                             

In [300]:
equity.to_csv('../../equity.csv')

In [301]:
meal_log[meal_log['hospital'] == 'Boston Medical Center']

,hospital,zone,priority_rank,vip,meal,quantity,restaurant,day,failed
23,Boston Medical Center,2,1,False,lunch_sat,150.0,LD,sat,False
24,Boston Medical Center,2,1,False,lunch_sun,150.0,NaN,sun,True
25,Boston Medical Center,2,1,False,dinner_mon,400.0,SU,mon,False
26,Boston Medical Center,2,1,False,dinner_tues,400.0,NaN,tues,True
27,Boston Medical Center,2,1,False,dinner_wed,400.0,NaN,wed,True
28,Boston Medical Center,2,1,False,dinner_thurs,400.0,NaN,thurs,True
29,Boston Medical Center,2,1,False,dinner_fri,400.0,NaN,fri,True
61,Boston Medical Center,2,1,True,lunch_mon,400.0,FC,mon,False
62,Boston Medical Center,2,1,True,lunch_tues,400.0,FC,tues,False
63,Boston Medical Center,2,1,True,lunch_wed,400.0,FC,wed,False


In [302]:
meal_log[meal_log['hospital'] == 'Massachusetts General Hospital - Central']

,hospital,zone,priority_rank,vip,meal,quantity,restaurant,day,failed
132,Massachusetts General Hospital - Central,3,1,True,lunch_tues,500.0,PG,tues,False
133,Massachusetts General Hospital - Central,3,1,True,lunch_thurs,500.0,PG,thurs,False
134,Massachusetts General Hospital - Central,3,1,True,lunch_sat,300.0,PG,sat,False
135,Massachusetts General Hospital - Central,3,1,True,dinner_mon,500.0,PG,mon,False
136,Massachusetts General Hospital - Central,3,1,True,dinner_wed,500.0,PG,wed,False
137,Massachusetts General Hospital - Central,3,1,True,dinner_fri,500.0,PG,fri,False
138,Massachusetts General Hospital - Central,3,1,True,dinner_sun,300.0,PG,sun,False


In [303]:
rest_usage.iloc[:, 2:].sum(axis=1).sort_values(ascending=False)

code
PG    4357.0
SH    3257.0
FC    2422.0
LD    2398.0
MD    1750.0
SU    1268.0
BB    1055.0
PO    1050.0
IG    1020.0
ME     581.0
RT     322.0
MM     312.0
CS     172.0
dtype: float64